In [58]:
import networkx as nx
import json, time
from IPython.display import HTML

graphsize = 100
nodes = {}
links = {}
Graph = nx.barabasi_albert_graph(graphsize, 1)
j = "{"    
j = j + "\t\"nodes\": ["
for n in range(0, graphsize):
    nodes[n] = {}
    nodes[n]['name'] = n
for n in nodes:
    j = j + str(json.dumps(nodes[n])) + ",\n"
j = j[:-2]
j = j + "\t],\n"
j = j + "\t\"links\":[\n"
for link in nx.edges(Graph):
    links[str(link)] = {}
    links[str(link)]['source'] = link[0]
    links[str(link)]['target'] = link[1]
for l in links:
    j = j + str(json.dumps(links[l])) + ",\n"
j = j[:-2]
j = j + "\t]\n"
j = j + "}"



In [59]:
html_code ="""
<html>
  <head>
    <script type="text/javascript">
        graph = %s;
    </script>
  </head>
  <body>
      <div id="graph"></div>
  </body>
</html>""" % (j)
HTML(html_code)

In [60]:
%%javascript

require.config({paths: {d3: "http://d3js.org/d3.v3.min",
                       }
               }
              );

require(["d3"], function(d3) {
        var width = 960,
        height = 500;
        var color = d3.scale.category10();

        var force = d3.layout.force()
            .charge(-120)
            .linkDistance(30)
            .size([width, height]);
        // append SVG to body of index.html with height and width specified above
        var svg = d3.select("div#graph").append("svg")
            .attr("width", width)
            .attr("height", height);

        // this actually adheres the data to the force base layout and starts the layout
        force
            .nodes(graph.nodes)
            .links(graph.links)
            .start();

        var link = svg.selectAll(".link").data(graph.links)
            .enter().append("line")
            .attr("class", "link")
            .style("stroke", "#999")
            .style("stroke-width", ".6px")
            .style("stroke-width", function(d) { return Math.sqrt(d.value); });

        // append the nodes with specified data and style properties
        var node = svg.selectAll(".node").data(graph.nodes)
            .enter().append("circle")
            .attr("class", "node")
            .attr("r", 5)
            .style("stroke", "#fff")
            .style("stroke-width", "1.5px")
            .style("fill", "steelblue")
            .call(force.drag);
        // attach titles to nodes, so when the mouse hovers over the nodes it projects the name
        node.append("title")
            .text(function(d) { return d.name; });

        // this is the main mechanism of the force based diagram
        // which moves the nodes and links from their starting and finishing positions
        // once it hits equilibrium, it will stop moving the positions
        force.on("tick", function() {
        link.attr("x1", function(d) { return d.source.x; })
            .attr("y1", function(d) { return d.source.y; })
            .attr("x2", function(d) { return d.target.x; })
            .attr("y2", function(d) { return d.target.y; });
        node.attr("cx", function(d) { return d.x; })
            .attr("cy", function(d) { return d.y; });
        });

});

<IPython.core.display.Javascript object>